In [106]:
'''
匯入套件
'''
# 操作 browser 的 API
from selenium import webdriver
from selenium.webdriver.chrome.service import Service
from webdriver_manager.chrome import ChromeDriverManager

# 處理逾時例外的工具
from selenium.common.exceptions import TimeoutException

# 面對動態網頁，等待某個元素出現的工具，通常與 exptected_conditions 搭配
from selenium.webdriver.support.ui import WebDriverWait

# 搭配 WebDriverWait 使用，對元素狀態的一種期待條件，若條件發生，則等待結束，往下一行執行
from selenium.webdriver.support import expected_conditions as EC

# 期待元素出現要透過什麼方式指定，通常與 EC、WebDriverWait 一起使用
from selenium.webdriver.common.by import By

# 強制等待 (執行期間休息一下)
from time import sleep

# 整理 json 使用的工具
import json

# 執行 command 的時候用的
import os

# 子處理程序，用來取代 os.system 的功能
import subprocess

'''
Selenium with Python 中文翻譯文檔
參考網頁：https://selenium-python-zh.readthedocs.io/en/latest/index.html
selenium 啓動 Chrome 的進階配置參數
參考網址：https://stackoverflow.max-everyday.com/2019/12/selenium-chrome-options/
Mouse Hover Action in Selenium
參考網址：https://www.toolsqa.com/selenium-webdriver/mouse-hover-action/
yt-dlp 下載影音的好工具
參考網址：https://github.com/yt-dlp/yt-dlp
'''

# 啟動瀏覽器工具的選項
my_options = webdriver.ChromeOptions()
# my_options.add_argument("--headless")                #不開啟實體瀏覽器背景執行
my_options.add_argument("--start-maximized")         #最大化視窗
my_options.add_argument("--incognito")               #開啟無痕模式
my_options.add_argument("--disable-popup-blocking") #禁用彈出攔截
my_options.add_argument("--disable-notifications")  #取消 chrome 推播通知
my_options.add_argument("--lang=zh-TW")  #設定為正體中文

# 使用 Chrome 的 WebDriver
driver = webdriver.Chrome(
    options = my_options,
    service = Service(ChromeDriverManager().install())
)

# 建立儲存圖片、影片的資料夾
folderPath = 'youtube'
if not os.path.exists(folderPath):
    os.makedirs(folderPath)

# 放置爬取的資料
listData = []

In [107]:
'''
以 function 名稱，作為爬蟲流程
'''
# 走訪頁面
def visit():
    driver.get('http://www.youtube.com/')
    
# 輸入關鍵字
def search():
    # 輸入名稱
    txtInput = driver.find_element(By.CSS_SELECTOR, 'input#search')
    txtInput.send_keys('張學友')
    
    # 等待
    sleep(1)
    
    # 送出表單
    txtInput.submit()
    
    # 等待
    sleep(1)
    
# 篩選(選項 Optional)
def filterFunc():
    try:
        # 等待篩選元素出現
        WebDriverWait(driver, 5).until(
            EC.presence_of_element_located(
                (By.CSS_SELECTOR, 'ytd-toggle-button-renderer.style-scope.ytd-search-sub-menu-renderer')
            )
        )
        
        # 按下篩選元素，使項目浮現
        driver.find_element(By.CSS_SELECTOR, 
            'ytd-toggle-button-renderer.style-scope.ytd-search-sub-menu-renderer'
        ).click()
        
        # 等待
        sleep(2)
        
        # 按下選擇的項目
        driver.find_elements(By.CSS_SELECTOR, 
            'yt-formatted-string.style-scope.ytd-search-filter-renderer'
        )[9].click()
        
        # 等待
        sleep(2)
        
#         driver.find_element(By.CSS_SELECTOR, 
#             'ytd-toggle-button-renderer.style-scope.ytd-search-sub-menu-renderer'
#         ).click()
        
#         sleep(2)
        
#         driver.find_elements(By.CSS_SELECTOR, 
#             'yt-formatted-string.style-scope.ytd-search-filter-renderer'
#         )[25].click()
        
#         sleep(2)
        
    except TimeoutException:
        print('等候逾時!')

# 滾動頁面
def scroll():
    '''
    innerHeight => 瀏覽器內部的高度
    offset => 當前捲動的量(高度)
    count => 累計無效滾動次數
    limit => 最大無效滾動次數
    '''
    innerHeight = 0
    offset = 0
    count = 0
    limit = 3
    
    # 在捲動到沒有元素動態產生前，持續捲動
    while count <= limit:
        # 每次移動高度
        offset = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        
        '''
        或是每次只滾動一點距離，
        以免有些網站會在移動長距離後，
        將先前移動當中的元素隱藏

        例如將上方的 script 改成:
        offset += 600
        '''
        
        # 捲軸往下滑動
        driver.execute_script(f'''
            window.scrollTo({{
                top: {offset},
                behavior: 'smooth'
            }});
        ''')
        
        # 等待
        sleep(3)
        
        # 透過執行 js 語法來取得捲動後的當前總高度
        innerHeight = driver.execute_script(
            'return window.document.documentElement.scrollHeight;'
        )
        
        # 經過計算，如果滾動距離(offset)大於等於視窗內部總高度(innerHeight)，代表已經到底了
        if offset == innerHeight:
            count += 1
            
        # 為了實驗功能，捲動超過一定的距離，就結束程式
        if offset >= 600:
            break
            
# 分析頁面元素資訊
def parse():
    # 取得主要元素的集合
    ytd_video_renderers = driver.find_elements(
        By.CSS_SELECTOR, 
        'ytd-video-renderer.style-scope.ytd-item-section-renderer'
    )
    
    # 逐一檢視元素
    for ytd_video_renderer in ytd_video_renderers:
        # 印出分隔線
        print("=" * 50)
        
        # 取得圖片連結
        img = ytd_video_renderer.find_element(
            By.CSS_SELECTOR,
            'ytd-thumbnail.style-scope.ytd-video-renderer img#img'
        )
        imgSrc = img.get_attribute('src')
        print(imgSrc)
        
        # 取得 a 連結
        a = ytd_video_renderer.find_element(
            By.CSS_SELECTOR,
            'a#video-title'
        )
        aTitle = a.get_attribute('innerText')
        print(aTitle)
        aLink = a.get_attribute('href')
        print(aLink)
        
        # 取得影音 ID
        strDelimiter = ''
        if 'shorts' in aLink:
            strDelimiter = '/shorts/'
        else:
            strDelimiter = 'v='
        
        youtube_id = aLink.split(strDelimiter)[1]
        print(youtube_id)
        
        # 放資料到 list 中
        listData.append({
            'id': youtube_id,
            'title': aTitle,
            'link': aLink,
            'img': imgSrc
        })
        
# 將 list 存成 json
def saveJson():
    with open(f'{folderPath}/youtube.json', 'w', encoding='utf-8') as file:
        file.write(json.dumps(listData, ensure_ascii=False, indent=4))
        # ensure_ascii=False : 英數字+中文
        # ensure_ascii=False : 英數字+/...
        # indent : 縮排
        
# 關閉瀏覽器
def close():
    driver.quit()

In [108]:
if __name__ == '__main__':
    visit()
    search()
    filterFunc()
    scroll()
    parse()
    saveJson()

https://i.ytimg.com/vi/XNddVxU3Fzc/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLA7-tGmw5qNrSoFUp7xFV1iUuAhyA
如果這都不算愛 - 張學友
https://www.youtube.com/watch?v=XNddVxU3Fzc
XNddVxU3Fzc
https://i.ytimg.com/vi/1bDAC3gmO4A/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCiZ3FbvElvWlEUZ0mahGyBXdekFw
張學友 - 我真的受傷了
https://www.youtube.com/watch?v=1bDAC3gmO4A
1bDAC3gmO4A
https://i.ytimg.com/vi/X9jBE0mGhf8/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLCI3AAnRYcSwMODi52BaPdXZEnXsg
最近很火的《不该用情》张学友版本，也太好听了吧
https://www.youtube.com/watch?v=X9jBE0mGhf8
X9jBE0mGhf8
https://i.ytimg.com/vi/yBctNUyCLZc/hqdefault.jpg?sqp=-oaymwEcCOADEI4CSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLDYJ3Mctm1CIryvgvwOoZIVq-j80g
張學友 Jacky Cheung & 梅艷芳 Anita Mui ─ 相愛很難【歌詞】
https://www.youtube.com/watch?v=yBctNUyCLZc
yBctNUyCLZc
https://i.ytimg.com/vi/JgVmmVxEYzk/hq720.jpg?sqp=-oaymwEcCOgCEMoBSFXyq4qpAw4IARUAAIhCGAFwAcABBg==&rs=AOn4CLBVA1JvnePbT5dH8hC8Xpqr

In [109]:
close()

In [112]:
'''
yt-dlp 專案
https://github.com/yt-dlp/yt-dlp

下載 yt-dlp (Linux 和 MacOS 版本，要設定「chmod +x yt-dlp」，MacOS 也要改成 yt-dlp)
- Windows: https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp.exe
- Linux: https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp
- MacOS: https://github.com/yt-dlp/yt-dlp/releases/latest/download/yt-dlp_macos

輸出檔案名稱範例
https://github.com/yt-dlp/yt-dlp#output-template-examples
'''
# 下載
def download():
    # 開啟 json
    with open(f'{folderPath}/youtube.json', 'r', encoding='utf-8') as file:
        # 取得 json 字串
        strJson = file.read()
    
    # 將 json 轉成 list
    listResult = json.loads(strJson)
    
    # 下載所有檔案
    for index, obj in enumerate(listResult):
        if index == 3:
            break
            
        print('=' * 50)
        print(f'正在下載: {obj["link"]}')
        
        # 定義指令
        cmd = [
            './yt-dlp.exe',
            obj['link'],
            '-f', 'b[ext=mp4]',
            '-o', f'{folderPath}/%(id)s.%(ext)s'
        ]
        
        # 執行指令，並取得回傳結果
        result = subprocess.run(
            cmd,
            stdout=subprocess.PIPE,
            stderr=subprocess.STDOUT
        )
        
        # 顯示實際執行過程的文字輸出
        output = result.stdout
        print('下載完成，訊息如下:')
        print(output)
        
'''另行下載'''
download()

正在下載: https://www.youtube.com/watch?v=XNddVxU3Fzc
下載完成，訊息如下:
b'[youtube] XNddVxU3Fzc: Downloading webpage\n[youtube] XNddVxU3Fzc: Downloading android player API JSON\n[info] XNddVxU3Fzc: Downloading 1 format(s): 18\n[download] Destination: youtube\\XNddVxU3Fzc.mp4\n\r[download]   0.1% of    1.64MiB at  Unknown B/s ETA Unknown\r[download]   0.2% of    1.64MiB at  Unknown B/s ETA Unknown\r[download]   0.4% of    1.64MiB at  Unknown B/s ETA Unknown\r[download]   0.9% of    1.64MiB at  Unknown B/s ETA Unknown\r[download]   1.8% of    1.64MiB at  Unknown B/s ETA Unknown\r[download]   3.8% of    1.64MiB at   15.43MiB/s ETA 00:00  \r[download]   7.6% of    1.64MiB at   20.73MiB/s ETA 00:00\r[download]  15.2% of    1.64MiB at   22.70MiB/s ETA 00:00\r[download]  30.4% of    1.64MiB at   29.43MiB/s ETA 00:00\r[download]  60.9% of    1.64MiB at   32.11MiB/s ETA 00:00\r[download] 100.0% of    1.64MiB at   32.98MiB/s ETA 00:00\r[download] 100% of    1.64MiB in 00:00:00 at 18.59MiB/s  \n'
正在下載: http